In [1]:
# system tools
import warnings
import json
import sys
import string
import ast

# data cleaning + analysis tools
import pandas as pd
import datetime as dt
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

#nltk tools
import spacy
import lda #Latent Dirichlet Allocation (create topics)
import gensim
from gensim import corpora, models #for constructing document term matrix
#from stop_words import get_stop_words
from gensim.models import Phrases
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.util import ngrams

#set notebook preferences
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)

Populating the interactive namespace from numpy and matplotlib


### Import JSON file with city metadata 

This including which cities have published the raw Public Record Requests (PRRs) they receive for analysis. 

In [21]:
json_file = '../data/cities.json'

with open(json_file, 'r') as f:
    md = json.load(f)

###  Create dataframe of PRR data for all relevant cities

This dataframe includes PPR data from the **33 cities** in our sample (52 total cities) that had sufficient raw data with the public record request for analysis. Our sample represents cities that host an online PRR portal for submitting requests.  These data were obtained through a variety of methods including:

1. exporting the full archive of PRRs hosted on the online portal as a csv file  
2. scraping the full history of PRR data from portals which publish previous requests, but do not offer a download option (see [scraping notebook](https://github.com/sunlightpolicy/Sunlight_FOIA/blob/master/src/data/NR_Scrape.ipynb))
3. downloading public records request data that has been published on city’s open data portal  
4. submitting a public record request to obtain the archive of PRR data 

It is worth noting that *specificities of the different city portals influence the substance of the public record requests received*. For example, the city of Clearwater, FL has separate request forms for police records and public records, prompting citizens who submit police record requests to provide the specific case number. In addition, while most of the data released by cities is the raw request submitted by citizens, in a few cases the city released a summary of the submitted request prepared by city staff. For example, the Oklahoma City clerk's office released the summary of the request and the department the request was routed to for response. 


In [23]:
data_raw = pd.DataFrame(columns = ['city', 'month_year', 'Summary'])
city_list = []
for key, value in md.items():
    city = value['name']
    filepath = '/Users/alenastern/Google Drive File Stream/My Drive/Alena_Project/PR_Data/{}.csv'.format(city)
    # tag in metadata for whether city publishes request content
    if value["desc"] == "Y":
        try:
            df = pd.read_csv(filepath)
        except:
            try:
                df = pd.read_csv(filepath, encoding='mac_roman')
            except:
                continue
        print(key)
        name = key.split(' ')
        city_list.extend([x for x in name[:-1]])
    else:
        continue
    
    try:
        df['Create Date'] = pd.to_datetime(df['Create Date'])
    except:
        df['New'] = pd.to_datetime(df['Create Date'].apply(lambda x: re.findall('^\S*', x)[0]))
        df.drop(columns=['Create Date'], inplace = True)
        df.rename(index=str, columns={"New": "Create Date"}, inplace = True)

    df['month_year'] = df['Create Date'].dt.to_period('M')
    
    mc = df[['month_year', 'Summary']]
    mc['city'] = city
    
    data_raw = pd.concat([data_raw, mc])


Arlington city
Bainbridge Island city
Boulder County
Cathedral City city
Clearwater city
Dayton city
Denton city
Everett city
Fort Collins city
Greensboro city
Hayward city
Kirkland city
Las Cruces city
Lynnwood city
Mercer Island city
Miami city
Middleborough town
New Orleans city
Oakland city
Oklahoma City city
Olympia city
Palo Alto city
Peoria city
Pullman city
Rancho Cucamonga city
Redmond city
Renton city
Sacramento city
San Francisco city
Tukwila city
Vallejo city
West Sacramento city
Winchester city


#### Our raw dataset includes 110,063 PRRs from 33 different cities

In [24]:
data_raw.to_csv('data_raw.csv')
data_raw.shape

(110063, 3)

In [25]:
len(data_raw.city.unique())

33

In [26]:
# create sequential numeric index for data

data_raw.index = pd.RangeIndex(len(data_raw.index))
data_raw.reset_index(inplace=True)

### Create dataframe for cleaning by removing null summaries

In [27]:
# Drop observations that are null for the raw PRR content field ('Summary')
data = data_raw.dropna(subset=['Summary'])

We see the raw data below. As we can see, the text in the Summary field is very messy and will require a lot of cleaning to prepare the data for analysis!

In [28]:
data.head()

,index,Summary,city,month_year
0,0,"We are working with an engineering firm on an upcoming project. They have asked us to gather maps for this project. Would you be able to assist me in gathering maps/records (as builds) for any underground water facilities you may have? Something just showing the route of the water lines would do.\n\n207th ST NE to 92nd Ave NE, Arlington, Cascade Surveying & Engineering \n\nI have attached the scope for your convenience. Please let me know if you have questions.",Arlington,2018-06
1,1,"Need copies of contracts and all related documents pertaining to Topcub Aircraft property located at 17922 59th DR NE Arlington WA 98223 between Arlington Airport, Topcub Aircraft, City of Arlington, HCI Steel Buildings and PUD.",Arlington,2018-06
2,2,"Copies of Building Permits of $5,000 valuation and up ($20,000 min for Re-Roofs), ($50,000 min. for Cell Tower upgrades), (Electrical, Mechanical & Plumbing at $100,000 min.) and (Solar Panels, Swimming Pools & Foundations at any valuation)",Arlington,2018-06
3,3,"police report filed to an officer against Wayne Parris (DOB 08-03-1957) from Brittany J. Parris. The paperwork I have has a case number D18-39 it is also stamped at the bottom with 18-1294, Iím not sure which number you will need. If there is any other information needed please let me know.",Arlington,2018-06
4,4,"Email Communications between Stephanie Shook, Dave Kraski, Bruce Stedman and Chad Schmidt in regards to Fire Protection District 21 billing and passage of contract for ALS Services. \n\nAlso any copies of Agenda Bills, D21 Contract and materials presented for review in Nov/Dec time frame in regards to the contract.",Arlington,2018-06


#### Function to convert nltk part of speech tags to wordnet tags (we use this to stem the words in data cleaning below):

In [29]:
def get_wordnet_pos(tag):

    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

#### A key challenge is removing proper names from the raw data. We initially tried using the NLTK and SpaCy proper noun tags, but ultimately found the best performance from using a dictionary of common first and last names. To create this dictionary, we first write a function to turn separate files of the 1000 most popular baby names by year provided by the [Social Security Administration](https://www.ssa.gov/OACT/babynames/) into a single set of unique first names across years


In [32]:
def name_list(st, end):
    names = set()
    for yr in range(st, end+1):
        fp = '../data/names/yob{}.txt'.format(yr)
        df = pd.read_table(fp, sep = ',', names = ['name', 'sex', 'count'])
        names |= set(df['name'])
    
    return list(names)

In [33]:
# create list of unique first names that were on the 1000 most popular names each year between 1950-2017

names = name_list(1950,2017)

#### Create list on common surnames in the United States. Data on surnames is from the U.S. Census Bureau, compiled by FiveThirtyEight and accessed via [data.world](https://data.world/fivethirtyeight/most-common-name/workspace/file?filename=README.md)

In [34]:
last_names = pd.read_csv('../data/names/surnames.csv')
last_names.name = last_names.name.str.title()
ln = list(last_names['name'])

In [35]:
# combine first names and surnames and create dictionary 

all_names = names + ln
all_names_dict = {key: 1 for key in all_names if key}

## Clean PRR data to prepare for LDA analysis

Prior to analysis, we clean our unstructured text data to improve the outcome of our LDA analysis results. Our goals are as follows:

1. Remove "noise" - words that do not provide information on the subject of a PRR (eg. stop words like "the", proper nouns like people's names or city names, punctuation and digits, and general words/phrase common to PRRs like "good morning" or "record"
2. Stem words so like words are treated as the same (eg. "photo" and "photos" should be regarded as the same word, as should "assault" and "assaulted"
3. Account for meaningful phrases where the combination of words has particular meaning (to avoid excessive computation time, we only consider two-word phrases)

In [39]:
# make a copy of the original request column to preserve
data['summary_raw'] = data['Summary']

# remove common public record request phrases - we remove as phrases because we care about specific combination/order 
# of words (we want to remove "open record request" not all instances of word "open")
phrase_list = ['public record request', 'open record request', 'public records request', 'open records request', 
               'foia request', 'see attached', 'see attachment', 'to whom it may concern', 'public records act',
              'electronic copy', 'electronic copies', 'freedom of information act', 'good afternoon', 'good morning',
              'good day']
                         
for phrase in phrase_list:
    s = re.compile(re.escape(phrase), re.IGNORECASE)
    data.Summary = data['Summary'].apply(lambda x: s.sub('', x))
    
    
# Replace common acronyms in Summary
data.Summary = data.Summary.str.replace('NOPD', 'police department')
data.Summary = data.Summary.str.replace('OPD' , 'police department')
data.Summary = data.Summary.str.replace('SFPD', 'police department')
data.Summary = data.Summary.str.replace('CPD', 'police department')
data.Summary = data.Summary.str.replace('APD', 'police department')
data.Summary = data.Summary.str.replace('GPD', 'police department')
data.Summary = data.Summary.str.replace('KPD', 'police department')
data.Summary = data.Summary.str.replace('TPD', 'police department')
data.Summary = data.Summary.str.replace('DPD', 'police department')
data.Summary = data.Summary.str.replace('EPD', 'police department')
data.Summary = data.Summary.str.replace('HPD', 'police department')
data.Summary = data.Summary.str.replace('LPD', 'police department')
data.Summary = data.Summary.str.replace('MDPD', 'police department')
data.Summary = data.Summary.str.replace('PPD', 'police department')
data.Summary = data.Summary.str.replace('SPD', 'police department')
data.Summary = data.Summary.str.replace('VPD', 'police department')
data.Summary = data.Summary.str.replace('CCPD', 'police department')
data.Summary = data.Summary.str.replace('FCPD', 'police department')
data.Summary = data.Summary.str.replace('TPD', 'police department')
data.Summary = data.Summary.str.replace('LCPD', 'police department')
data.Summary = data.Summary.str.replace('OKCPD', 'police department')
data.Summary = data.Summary.str.replace('PAPD', 'police department')
data.Summary = data.Summary.str.replace('RCPD', 'police department')
data.Summary = data.Summary.str.replace('WSPD', 'police department')

# PDRD = portable digital recording device (body cam) worn by police
data.Summary = data.Summary.str.replace('PDRD', 'police body camera')
data.Summary = data.Summary.str.replace('CPS', 'child protective services')

#https://www.sfdph.org/dph/EH/HMUPA/HMUPAFormsMenu.asp - hazardous materials
#https://www.waterboards.ca.gov/ust/contacts/docs/lop_guide.pdf - water resources local oversight program
data.Summary = data.Summary.str.replace('LOP', 'water')
data.Summary = data.Summary.str.replace('HMUPA', 'hazardous materials')

# Replace key numbers with strings
data.Summary = data.Summary.str.replace(' 911 ', ' nineoneone ')
data.Summary = data.Summary.str.replace(' 311 ', ' threeoneone ')
data.Summary = data.Summary.str.replace(' 9-11 ', ' nineoneone ')
data.Summary = data.Summary.str.replace(' 3-11 ', ' threeoneone ')

# Remove digits
dig_translator = str.maketrans('','', string.digits)
data.Summary = data.Summary.str.translate(dig_translator)

# because "will" is in the NLTK list of stopwords below, we treat 'final will' separately                         
c = re.compile(re.escape('final will'), re.IGNORECASE)
data.Summary = data['Summary'].apply(lambda x: s.sub('final final_will', x))

# replace hyphen and slash with space to treat hyphate words as two separate words
hyphen_translator = str.maketrans('-/','  ')
data.Summary = data.Summary.str.translate(hyphen_translator)

# remove all punctuation
translator = str.maketrans('','', string.punctuation)
data.Summary = data.Summary.str.translate(translator)

# split text into list of words by space 
data['token'] = data['Summary'].apply(lambda x: nltk.word_tokenize(x))

# remove proper first and last names in our dictionary + convert all words to lower case
data['token'] = data['token'].apply(lambda x: [i.lower() for i in x if i not in all_names_dict])

#remove empty strings, stopwords and stem
stop_words = set(stopwords.words('english'))
lmtzr = WordNetLemmatizer()
data['lemma'] = data['token'].apply(lambda x: nltk.pos_tag(x))
data['mash'] = data['lemma'].apply(lambda x: [lmtzr.lemmatize(i[0], get_wordnet_pos(i[1])) for i in x if len(i[0]) > 0 and i[0] not in stop_words])

# Remove whitespace
wsp_translator = str.maketrans('','', string.whitespace)
data['mash'] = data['mash'].apply(lambda x: [i.translate(wsp_translator) for i in x])

# Remove empty lists
data['mash_len'] = data['mash'].apply(lambda x: len(x))
data = data[data['mash_len'] > 0]


In [40]:
# remove number suffixes
suffix_list = ['th', 'nd', 'st', 'rd', 'blvd', 'pkwy']
data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in suffix_list])

# remove city and state abbreviations
abbv_list = ['wa', 'nc', 'co', 'ca', 'oh', 'tx', 'nm', 'fl', 'ma', 'la', 'ok', 'az', 'ri', 'va', 
             'francisco', 'sf', 'okc', 'lv', 'nola', 'slc', 'cw']
data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in abbv_list])

# remove spelled numbers
num_list = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']
data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in  num_list])

# replace 'inc' with 'incident
data['mash'] = data['mash'].apply(lambda x: ['incident' if i=='inc' else i for i in x])

# replace 'pd' with 'police department
data['mash'] = data['mash'].apply(lambda x: ['police department' if i=='pd' else i for i in x])

# remove noise words
noise = ['dr', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sept', 'sep', 'oct', 'nov', 'dec', 
        'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 
        'december', 'ne', 'nw', 'se', 'sw', 'ct', 'dr', 'way', 'dv', 'ave', 'aka', 'get', 'look', 'im', 'want', 
        'find', 'could', 'go', 'take', 'e', 'n', 's', 'w', '“', '’', '”', '•', 'northeast', 'northwest', 'southeast', 
        'southwest', 'north', 'south', 'east', 'west', 'orleans', '–', 'a', 'b', 'c', 'd', 'f', 'g', 'h', 'i', 'j', 'k',
        'l', 'm', 'o', 'p', 'q', 'r', 't', 'u', 'v', 'x', 'y', 'z', 'am', 'pm', 'hr', 'mr', 'ms', 'mrs', 'johnson', 
        'jr', 'kent', 'christopher', 'miller', 'joe', 'willows', 'david', 'michael', 'john', 'red', 'robert',
        'ask', 'able', 'let', 'question', 'also', 'snohomish', '¬ß', 'per', 'available', 'test', '√Ø', 'andor', '·', 'etc',
        'ï', 'ce', 'eg', 'sammamish']

data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in noise])

# remove cities
city_list = ['arlington', 'asheville', 'bainbridge', 'island', 'boulder', 'cathedral' ,'clearwater', 'dayton', 
            'denton', 'everett', 'fort', 'collins', 'greensboro', 'hayward', 'kirkland', 'las', 'cruces', 'lynnwood',
            'mercer', 'miami', 'middleborough', 'new', 'orleans', 'oakland', 'oklahoma', 'olympia', 'palo', 'alto', 
            'peoria', 'pullman', 'rancho', 'cucamonga', 'redmond', 'renton', 'sacramento', 'san', 'francisco', 
            'tukwila', 'vallejo', 'west', 'sacramento', 'winchester']

data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in city_list])

# Remove all state names
state_list = ['washington', 'carolina', 'colorado', 'california',
             'ohio', 'texas', 'florida', 'new', 'mexico','massachusetts',
             'louisiana', 'oklahoma', 'arizona', 'rhode', 'virginia']

data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in state_list])

# Create two-word phrases (bigrams)
data['bigrams'] = data['mash'].apply(lambda x: ["_".join(w) for w in ngrams(x, 2)])

#### Identify and remove noise words that are commonly used in PRRs

In [41]:
word_list = [y for x in list(data['mash']) for y in x]
counts = Counter(word_list)
Counter(word_list).most_common(50)

[('report', 43899),
 ('request', 31182),
 ('record', 23179),
 ('please', 21970),
 ('copy', 20125),
 ('property', 13058),
 ('information', 12020),
 ('number', 11451),
 ('provide', 10819),
 ('would', 10607),
 ('include', 10494),
 ('incident', 10264),
 ('document', 9837),
 ('police', 9587),
 ('permit', 9376),
 ('accident', 9038),
 ('thank', 8925),
 ('address', 8436),
 ('department', 8347),
 ('like', 8025),
 ('location', 7985),
 ('regard', 7839),
 ('date', 7570),
 ('email', 7463),
 ('relate', 6769),
 ('reference', 6760),
 ('building', 6719),
 ('insurance', 6603),
 ('call', 6573),
 ('case', 6549),
 ('following', 6439),
 ('video', 6264),
 ('type', 5770),
 ('violation', 5628),
 ('file', 5622),
 ('plan', 5469),
 ('auto', 5393),
 ('need', 5212),
 ('dob', 4978),
 ('name', 4857),
 ('insure', 4774),
 ('public', 4498),
 ('transaction', 4476),
 ('city', 4416),
 ('make', 4366),
 ('occurrence', 4360),
 ('project', 4336),
 ('list', 4318),
 ('involve', 4190),
 ('time', 4119)]

We see that general words like "report", "please", and "copy" ae very common, yet do not provide information on the content of the request. We remove these common and general words, keeping common words that provide information like "property" and "police".

In [42]:
common_list = ['report', 'request', 'record', 'city', 'please', 'copy', 'date', 'information', 'would', 'regard', 'public',
              'include', 'document', 'provide', 'like', 'thank', 'need', 'know', 'thanks', 'pursuant', 'dear', 'file',
              'relate', 'from', 'either', 'hello', 'hi', 'foia', 'requestors', 'requestor', 'receive', 'available', 
               'make', 'attach', 'pertain', 'might', 'see', 'near']

# remove general words that are common to public record requests
data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in common_list])

#### Identify meaningful phrases by looking at the list of two-word sequences (bigrams) that are frequently used in public record requests. The meaningful phrases that we identify will be added to the list of words to consider in analysis for the PRRs in which they appear.

In [43]:
bigram_list = [y for x in list(data['bigrams']) for y in x]
counts = Counter(bigram_list)
Counter(bigram_list).most_common(100)

[('would_like', 7167),
 ('please_provide', 6801),
 ('accident_report', 5311),
 ('police_report', 4861),
 ('report_number', 4851),
 ('report_type', 4341),
 ('transaction_reference', 4315),
 ('auto_accident', 4299),
 ('occurrence_location', 4227),
 ('reference_insurance', 4207),
 ('number_occurrence', 4036),
 ('type_auto', 3557),
 ('request_copy', 3259),
 ('incident_report', 2889),
 ('offense_report', 2493),
 ('like_request', 2180),
 ('code_violation', 2174),
 ('provide_copy', 2092),
 ('police_department', 1894),
 ('include_limited', 1732),
 ('public_record', 1705),
 ('property_locate', 1698),
 ('certificate_occupancy', 1693),
 ('copy_police', 1621),
 ('hazardous_material', 1609),
 ('request_record', 1563),
 ('building_permit', 1485),
 ('storage_tank', 1410),
 ('please_know', 1288),
 ('request_please', 1274),
 ('please_advise', 1223),
 ('please_send', 1198),
 ('document_relate', 1183),
 ('please_contact', 1164),
 ('law_enforcement', 1147),
 ('copy_report', 1102),
 ('collision_report', 10

While some of the common bigrams do not provide insight (such as "would_like" and "please_provide") others provide key meaning that is not captured by the individual words such as "police_report", "auto_accident", and "building_permit". We add the meaningful bigrams in the list below to the final set of words for the PRRs in which they occur. 

In [44]:

common_bigrams = ['police_report', 'insurance_company', 'location_loss', 'date_occurrence', 'reportcase_number',
                  'insure_driver', 'auto_accident', 'occurrence_location', 'transactionreference_insurance', 'number_date', 'type_auto',
                  'accident_reportcase', 'code_violation', 'copy_police', 'incident_report', 'police_department', 'certificate_occupancy',
                  'accident_report', 'property_locate', 'storage_tank','driver_note', 'building_permit', 'driver_driver','case_number', 
                  'hazardous_material', 'collision_report', 'state_farm', 'site_plan', 'fire_department', 'ftp_report', 'auto_theft',
                  'fire_code', 'request_police', 'farm_claim', 'claim_compass', 'site_assessment', 'compass_report', 'environmental_site', 
                  'tax_sale', 'loss_cross','city_council', 'code_enforcement', 'subject_property', 'report_case', 'phase_environmental', 
                  'report_incident', 'date_loss', 'police_case', 'witness_statement', 'driving_record', 'break_in', 'birth_certificate', 
                  'death_certificate', 'background_check', 'public_works', 'lease_agreement', 'medical_record', 'billing_record', 
                  'record_check', 'records_check', 'marriage_certificate', 'marriage_record', 'park_ticket', 'miss_person',
                 'marriage_license', 'reckless_driving', 'arrest_report', 'medical_billing', 'medical_report', 'criminal_record',
                 'floor_plan', 'site_plan', 'building_plan', 'building_code', 'code_enforcement', 'personnel_file']

data['common_bigrams'] = data['bigrams'].apply(lambda x: [i for i in x if i in common_bigrams])

#### Combine columns containing cleaned words (mash) and meaningful phrases (common_bigrams) to yield final set of words for analysis for each PRR

In [45]:
data['final_mash'] = data['mash'] + data['common_bigrams']

# Remove empty lists
data['mash_len'] = data['final_mash'].apply(lambda x: len(x))
data = data[data['mash_len'] > 0]

#### We can see the result of the final cleaned data below. 
The final_mash column represents the set of words that will be considered in our analysis.

In [46]:
data.head()

,index,Summary,city,month_year,token,lemma,mash,mash_len,bigrams,common_bigrams,final_mash
0,0,We are working with an engineering firm on an upcoming project They have asked us to gather maps for this project Would you be able to assist me in gathering maps records as builds for any underground water facilities you may have Something just showing the route of the water lines would do\n\nth ST NE to nd Ave NE Arlington Cascade Surveying Engineering \n\nI have attached the scope for your convenience Please let me know if you have questions,Arlington,2018-06,"[are, working, with, an, engineering, firm, on, an, upcoming, project, they, have, asked, us, to, gather, maps, for, this, project, would, you, be, able, to, assist, me, in, gathering, maps, records, as, builds, for, any, underground, water, facilities, you, may, have, something, just, showing, the, route, of, the, water, lines, would, do, th, st, ne, to, nd, ne, cascade, surveying, engineering, i, have, attached, the, scope, for, your, convenience, please, let, me, know, if, you, have, questions]","[(are, VBP), (working, VBG), (with, IN), (an, DT), (engineering, NN), (firm, NN), (on, IN), (an, DT), (upcoming, JJ), (project, NN), (they, PRP), (have, VBP), (asked, VBN), (us, PRP), (to, TO), (gather, VB), (maps, NNS), (for, IN), (this, DT), (project, NN), (would, MD), (you, PRP), (be, VB), (able, JJ), (to, TO), (assist, VB), (me, PRP), (in, IN), (gathering, VBG), (maps, NNS), (records, NNS), (as, IN), (builds, NNS), (for, IN), (any, DT), (underground, JJ), (water, NN), (facilities, NNS), (you, PRP), (may, MD), (have, VB), (something, NN), (just, RB), (showing, VBG), (the, DT), (route, NN), (of, IN), (the, DT), (water, NN), (lines, NNS), (would, MD), (do, VB), (th, VB), (st, VB), (ne, JJ), (to, TO), (nd, VB), (ne, JJ), (cascade, NN), (surveying, VBG), (engineering, NN), (i, NN), (have, VBP), (attached, VBN), (the, DT), (scope, NN), (for, IN), (your, PRP$), (convenience, NN), (please, NN), (let, VB), (me, PRP), (know, VB), (if, IN), (you, PRP), (have, VBP), (questions, NNS)]","[work, engineering, firm, upcoming, project, gather, map, project, assist, gather, map, build, underground, water, facility, something, show, route, water, line, cascade, survey, engineering, scope, convenience]",25,"[work_engineering, engineering_firm, firm_upcoming, upcoming_project, project_gather, gather_map, map_project, project_would, would_assist, assist_gather, gather_map, map_record, record_build, build_underground, underground_water, water_facility, facility_something, something_show, show_route, route_water, water_line, line_would, would_cascade, cascade_survey, survey_engineering, engineering_attach, attach_scope, scope_convenience, convenience_please, please_know]",[],"[work, engineering, firm, upcoming, project, gather, map, project, assist, gather, map, build, underground, water, facility, something, show, route, water, line, cascade, survey, engineering, scope, convenience]"
1,1,Need copies of contracts and all related documents pertaining to Topcub Aircraft property located at th DR NE Arlington WA between Arlington Airport Topcub Aircraft City of Arlington HCI Steel Buildings and PUD,Arlington,2018-06,"[copies, of, contracts, and, all, related, documents, pertaining, to, topcub, aircraft, property, located, at, th, dr, ne, wa, between, airport, topcub, aircraft, of, hci, buildings, and, pud]","[(copies, NNS), (of, IN), (contracts, NNS), (and, CC), (all, DT), (related, JJ), (documents, NNS), (pertaining, VBG), (to, TO), (topcub, VB), (aircraft, NN), (property, NN), (located, VBN), (at, IN), (th, NN), (dr, NN), (ne, JJ), (wa, NN), (between, IN), (airport, NN), (topcub, NN), (aircraft, NN), (of, IN), (hci, NN), (buildings, NNS), (and, CC), (pud, NN)]","[contract, related, topcub, aircraft, property, locate, airport, topcub, aircraft, hci, building, pud]",13,"[copy_contract, contract_related, related_document, document_pertain, pertain_topcub, topcub_aircraft, aircraft_property, property_loca

After cleaning, we have 89,145 observations (this means that just over 10,000 PRRs contained no information that could be used for analysis. For example, a PRR that is just a person's name or an alphanumeric case number). We see below that the average PRR has just over 13 words in the final mash. It is worth noting that there are a significant number of PRRs that yield a final mash of only 1 word. As we test various LDA models, we will test a number of different additional criteria to exclude records with limited information from analysis to determine whether the performance of our models improves.

In [47]:
data['mash_len'].describe()

count    89145.000000
mean        13.144114
std         24.484836
min          1.000000
25%          3.000000
50%          6.000000
75%         15.000000
max       2512.000000
Name: mash_len, dtype: float64

#### We can see that there is significant variation in the average request length per city. 
In some cases, cities with short average length represent cities where the provided data represented a summary of the original request (Oklahoma City) though in other cases, like Dayton, we received the raw data and the average length is still considerably shorter than other cities. 

In [48]:
data_gp = data.groupby('city').mean()
data_gp['mash_len']

city
Arlington          11.429670
Bainbridge         17.887482
Boulder            23.303030
CathedralCity      12.507463
Clearwater          7.233235
Dayton              2.617308
Denton             20.767620
Everett            13.735452
FortCollins        64.732143
Greensboro          2.329954
Hayward            32.089041
Kirkland           10.635076
LasCruces          26.474926
Lynnwood           16.546875
Mercer             16.500000
Miami              31.863489
Middleborough      26.000000
Nola               27.552862
OKC                 3.289256
Oakland            16.951460
Olympia            11.122305
PaloAlto           31.666667
Peoria              7.336830
Pullman             8.857691
RanchoCucamonga    22.843750
Redmond            12.497893
Renton             18.460102
Sacramento          4.783000
SanFrancisco       25.152969
Tukwila             6.114954
Vallejo            35.956268
WestSacramento     21.626126
Winchester         23.638418
Name: mash_len, dtype: float64

#### We can see a couple of examples of the cleaned mash and the original request:

In [49]:
data['Summary'][data.index == 164]

164    Police Traffic Collision Report and any other police reports records or documents relating to \nCase     \n\nUnit    Carla Jaramillo\nUnit    Jordan Boss\n\nTime of incident   
Name: Summary, dtype: object

In [50]:
data['final_mash'][data.index == 164]

164    [traffic, collision, police, unit, unit, time, incident, collision_report, police_report]
Name: final_mash, dtype: object

In [51]:
data['final_mash'][data.index == 60000]

60000    [display, bidder, individual, line, item, bid, price, youth, job, award, bid, proposal]
Name: final_mash, dtype: object

In [52]:
data["Summary"][60000]

'I would like a copy of\xa0the records displaying the bidders and their individual line item bid prices for the Youth Study Center Streets job that was awarded bid proposal no C  Thank you'

#### Save the cleaned data to csv to use for testing different LDA models

In [53]:
data.to_csv('data.csv', index=False)

Next, we tested a number of different parameters for the LDA models to identify the optimal model. Because these models are very computationally intensive and take a long time to run, we have included the tests and final model in a [separate notebook](https://github.com/sunlightpolicy/Sunlight_FOIA/blob/master/src/analysis/LDA_Model_Tests.ipynb). Below, we conduct the analysis on our final model.

### We write a few functions that will help us to analyze the results of our winning model:

In [2]:
def highest_topic(fp):
    # Identifies the topic that has received the highest composition score for each PRR 
    df = pd.read_csv(fp)
    df['topic_comp'] =  df['topic_comp'].apply(lambda x:  ast.literal_eval(x))
    df['comp_len'] = df['topic_comp'].apply(len)
    df = df[df['comp_len'] > 0]
    df['top_topic'] = df['topic_comp'].apply(lambda x: max(x, key=lambda item:item[1])[0])
    df['top_topic_comp'] = df['topic_comp'].apply(lambda x: max(x, key=lambda item:item[1])[1])
    
    return df

In [8]:
def topics_to_csv(df, num_topics):
    # Creates a csv file with the PRRs that have been assigned to each highest topic in the number of topics given 
    for topic in range(0, num_topics):
        subset = df[df['top_topic'] == topic]
        file_name = 'topics/{}_PRR_topic_{}.csv'.format(num_topics, topic)
        subset.to_csv(file_name)

In [ ]:
def process_csv(model_list):
    # Assigns highest topics and creates individual topic csv files
    for model in model_list:
        fp = 'topics/lda_{}_45_topics.csv'.format(model)
        df = pd.read_csv(fp)
        highest_topic(df, fp)
        topics_to_csv(df, model)
    

### Load Best Model and Data

Now that we have identified the best model to categorize the PRRs in our sample into coherent topics, we load the best model and view the 60 topics that the model has generated. 

In [5]:
# The best model grouped PRRs into 60 topics and imposed the requirement that the length of the final mash be at least 
# words long or the sum of the frequency count of all the words in the final mash be at least 2000.

final_model = gensim.models.ldamodel.LdaModel.load('lda_data_c2000_3_60')
final_fp = 'topics/lda_data_c2000_3_60.csv'
final_model.show_topics(num_topics=60, formatted=False)

[(0,
  [('complaint', 0.1985935),
   ('log', 0.14190955),
   ('phone', 0.08581352),
   ('search', 0.083066285),
   ('warrant', 0.03568498),
   ('estate', 0.032001145),
   ('involved', 0.026251603),
   ('quality', 0.02599625),
   ('real', 0.025753625),
   ('ensure', 0.019929796)]),
 (1,
  [('associate', 0.14829391),
   ('client', 0.11122169),
   ('photograph', 0.10852045),
   ('office', 0.08708323),
   ('follow', 0.061714146),
   ('emergency', 0.042823374),
   ('transfer', 0.033178225),
   ('color', 0.024401158),
   ('restaurant', 0.02243146),
   ('time', 0.018097397)]),
 (2,
  [('name', 0.15037733),
   ('victim', 0.084349126),
   ('crime', 0.082644366),
   ('officer', 0.07667519),
   ('domestic', 0.041578013),
   ('violence', 0.03346935),
   ('injury', 0.030954132),
   ('capitol', 0.030555379),
   ('location', 0.027925178),
   ('type', 0.021750417)]),
 (3,
  [('permit', 0.2758547),
   ('issue', 0.11701758),
   ('building', 0.07663683),
   ('build', 0.06414055),
   ('building_permit', 0

We can see that some of the topics above are grouped based on different words that people use to describe similar information. For example, topic 19 represents PRRs requesting incident information about auto thefts and topic 2 represents PRRs requesting incident information about general crimes, with an emphasis on domestic violence. While it's interesting to understand the demand for information about different types of crime, for cities looking to make decisions about what information to release as open data, these two topics refer to the same type of data - police incident reports. Therefore, to provide our city partners with information about what data types are requested, we group each of the 60 topics generated by the model based upon the type of data that would satisfy the request. The resulting 19 data type categories are listed in the dictionary below.

19,[('theft', 0.22061245),
   ('along', 0.06716178),
   ('auto_theft', 0.063398145),
   ('parking', 0.062286988),
   ('auto', 0.04646996),
   ('pl', 0.043082036),
   ('southcenter', 0.035162725),
   ('lot', 0.024039797),
   ('main', 0.020844543),
   ('university', 0.018294595)])
   
2,[('name', 0.15037733),
   ('victim', 0.084349126),
   ('crime', 0.082644366),
   ('officer', 0.07667519),
   ('domestic', 0.041578013),
   ('violence', 0.03346935),
   ('injury', 0.030954132),
   ('capitol', 0.030555379),
   ('location', 0.027925178),
   ('type', 0.021750417)]),

In [6]:
data_type_dict = {0: 'Complaints to city',
                  1: 'Police incident report',
                  2: 'Police incident report',
                  3: 'Parcel records, permits, plans',
                  4: '911/law enforcement service calls',
                  5: 'Public works, utilities',
                  6: 'Parcel records, permits, plans',
                  7: 'Property liens',
                  8: 'Purchasing records, contracts',
                  9: 'Criminal record check',
                  10: 'Criminal record check',
                  11: 'Purchasing records, contracts',
                  12: 'Criminal record check',
                  13: '911/law enforcement service calls',
                  14: 'City government meeting notes',
                  15: 'Checks and deposits',
                  16: 'Auto collision report',
                  17: 'Parcel records, permits, plans',
                  18: 'Crime photo and video',
                  19: 'Police incident report',
                  20: 'Employee benefits, payroll',
                  21: 'Criminal record check',
                  22: 'Auto collision report',
                  23: 'Human services cases',
                  24: 'Environmental assessment, hazardous materials',
                  25: 'Auto collision report',
                  26: 'City emails, social media posts',
                  27: 'Uncategorized',
                  28: 'Police incident report',
                  29: 'Property liens',
                  30: 'Building code violations',
                  31: 'Crime photo and video',
                  32: 'Uncategorized',
                  33: 'Auto collision report',
                  34: 'Police incident report',
                  35: 'City emails, social media posts',
                  36: 'Police incident report',
                  37: 'Criminal record check',
                  38: 'Property liens',
                  39: 'Employee benefits, payroll',
                  40: 'Crime photo and video',
                  41: 'Auto collision report',
                  42: 'Uncategorized',
                  43: 'Parcel records, permits, plans',
                  44: 'Parcel records, permits, plans',
                  45: 'Criminal record check',
                  46: 'Parcel records, permits, plans',
                  47: 'Uncategorized',
                  48: 'Police incident report',
                  49: 'Criminal record check',
                  50: 'Uncategorized',
                  51: 'Public works, utilities',
                  52: 'Uncategorized',
                  53: 'Police incident report',
                  54: 'Uncategorized',
                  55: 'Environmental assessment, hazardous materials',
                  56: 'Police incident report',
                  57: 'Public works, utilities',
                  58: 'Witness statements',
                  59: 'Police incident report'}

In [7]:
# Identify the top topic for each PRR and assign each PRR to a data type based on the top topic

final_data = highest_topic(final_fp)
final_data['data_type'] = final_data['top_topic'].apply(lambda x: data_type_dict[x])

In [8]:
clean_data = pd.read_csv('../data/clean_data.csv')
cs = clean_data['city_x'].str.split(" ", expand = True)
clean_data['city'] = cs[0]

clean_data['my_city'] = clean_data['month_year']+clean_data['city']
clean_data = clean_data[['my_city', 'policy', 'portal', 'robust_policy', 'robust_portal', 'policy_months', 'portal_months']]

final_data['month_year'] = final_data['month_year']+'-01'
final_data['my_city'] = final_data['month_year'] + final_data['city']
final_data = final_data.merge(clean_data, how ='left', on = 'my_city')

final_data.to_csv('final_data.csv')

## Identify Most Popular Catgories

### "Winner take all" popularity metric:
* Scoring Rules:
    * Only the topic that composes the largest share of a document scores "points" for its "Adjusted Popularity" total.
    * If a topic composes the largest share of that document, its "points" are its composition score.


In [84]:
def winner_take_all(df, count, measure): 
    # categorize data by final model identify highest topics
    final_df = df[[measure, 'top_topic_comp']]
    if count:
        topic_gp = final_df.groupby(measure).count()
    else:
        topic_gp = final_df.groupby(measure).sum()
    topic_gp.reset_index(inplace = True)
    topic_gp.rename(index=str, columns={"top_topic_comp": "total_pop"}, inplace = True)
    topic_gp = topic_gp[[measure, 'total_pop']]

    if measure == 'top_topic':
        topic_gp.sort_values(by=['top_topic'], ascending = True)
        # add in topic words
        words_in_topics = [tup[1] for tup in final_model.show_topics(num_topics=60, formatted=False)] # update w/ winning model
        topic_gp['topic'] = words_in_topics

        topic_gp[["topic1", "topic2", "topic3", "topic4", 
               "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = topic_gp.topic.apply(pd.Series)

    topic_gp = topic_gp.sort_values(by = 'total_pop', ascending=False)
    topic_gp.reset_index(drop = True, inplace = True)
    return topic_gp

In [85]:
topic_gp_count = winner_take_all(final_data, True, 'top_topic')
topic_gp_sum = winner_take_all(final_data, False, 'top_topic')

In [65]:
topic_gp_count.to_csv('topics/final_model_wta_count.csv')
topic_gp_sum.to_csv('topics/final_model_wta_sum.csv')

In [87]:
wta_data_sum = winner_take_all(final_data, False, 'data_type')
wta_data_count = winner_take_all(final_data, True, 'data_type')

In [89]:
wta_data_sum.to_csv('topics/final_model_wta_count_data.csv')
wta_data_count.to_csv('topics/final_model_wta_sum_data.csv')

In [88]:
wta_data_sum

,data_type,total_pop
0,Police Incident Report,6056.105429
1,Auto Collision Report,5410.516739
2,"Parcel Records, Permits, Plans",3635.726240
3,Crime Photo and Video,3223.739908
4,Criminal Record Check,2514.669181
5,"Environmental Assessment, Hazardous Materials",1631.869153
6,"City Emails, Social Media",1352.116572
7,911/Law Enforcement Service Calls,1253.521343
8,Building Code Violations,1146.562057
9,Property Liens,968.321523


## "Winner Take All with Thresholds" Rules:

Scoring Rules:
* Same as "Winner Take All", except a winning topic must compose at least a certain threshold of a document to get any points.
* We'll try 0.2 (low) and 0.5 (high) thresholds.

In [95]:
def winner_thresh(df, thresh, count, measure):
    final_df = df[[measure, 'top_topic_comp']]


    final_df = final_df[final_df['top_topic_comp'] >= thresh]
    if count:
        topic_gp = final_df.groupby(measure).count()
    else:
        topic_gp = final_df.groupby(measure).sum()
        
    topic_gp.reset_index(inplace = True)
    topic_gp.rename(index=str, columns={"top_topic_comp": "total_pop"}, inplace = True)
    topic_gp = topic_gp[[measure, 'total_pop']]

    if measure == 'top_topic':
        topic_gp.sort_values(by=['top_topic'], ascending = True)

        # add in topic words
        words_in_topics = [tup[1] for tup in final_model.show_topics(num_topics=60, formatted=False)] # update w/ winning model
        topic_gp['topic'] = words_in_topics

        topic_gp[["topic1", "topic2", "topic3", "topic4", 
           "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = topic_gp.topic.apply(pd.Series)

    topic_gp = topic_gp.sort_values(by='total_pop', ascending=False)
    topic_gp.reset_index(drop = True, inplace = True)
    return topic_gp

In [97]:
topic2 = winner_thresh(final_data, 0.2, False, 'top_topic')
topic5 = winner_thresh(final_data, 0.5, False, 'top_topic')

In [96]:
topic2_dt = winner_thresh(final_data, 0.2, False, 'data_type')
topic5_dt = winner_thresh(final_data, 0.5, False, 'data_type')

In [98]:
topic2_dt

,data_type,total_pop
13,Police Incident Report,5826.818697
1,Auto Collision Report,5329.277310
12,"Parcel Records, Permits, Plans",3359.758105
7,Crime Photo and Video,3175.558612
8,Criminal Record Check,2371.972375
10,"Environmental Assessment, Hazardous Materials",1584.073551
4,"City Emails, Social Media",1221.631425
0,911/Law Enforcement Service Calls,1184.330523
2,Building Code Violations,1118.106614
16,"Purchasing Records, Contracts",886.408014


## Partial Credit Approach

* All topics assigned to a given PRR get credit for that PRR's topic composition score, provided the score is above the established threshold

In [12]:
def prop_calc(df, thresh, measure, data_type_dict = data_type_dict):
    results_dict = {}
    for row_num in df.index:
        for tup in df.topic_comp[row_num]: 
            if tup[1] >= thresh: 
                if measure == 'top_topic':
                    if not tup[0] in results_dict:
                        results_dict[tup[0]] = tup[1] 
                    else:
                         results_dict[tup[0]] += tup[1]
                else:
                    if not data_type_dict[tup[0]] in results_dict:
                        results_dict[data_type_dict[tup[0]]] = tup[1] 
                    else:
                         results_dict[data_type_dict[tup[0]]] += tup[1]
            else:
                pass
                        
    pd_df = pd.DataFrame.from_dict(results_dict, orient = 'index')
    pd_df.reset_index(inplace = True)
    pd_df.rename(index = str, columns = {'index': measure, 0: 'total_score'}, inplace = True)
    pd_df = pd_df.sort_values(by=[measure], ascending = True)

    if measure == 'top_topic':
        # add in topic words
        words_in_topics = [tup[1] for tup in final_model.show_topics(num_topics=60, formatted=False)] # update w/ winning model
        pd_df['topic_words'] = words_in_topics

        pd_df[["topic1", "topic2", "topic3", "topic4", 
               "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = pd_df.topic_words.apply(pd.Series)

    
    
    pd_df = pd_df.sort_values(by='total_score', ascending=False)
    pd_df.reset_index(drop = True, inplace = True)
    return pd_df

In [32]:
prop_calc_2 = prop_calc(final_data, .2, 'top_topic')
prop_calc_5 = prop_calc(final_data, .5, 'top_topic')

In [33]:
prop_calc_2.to_csv("topics/prop_calc_2_final_model.csv")
prop_calc_5.to_csv("topics/prop_calc_5_final_model.csv")

In [34]:
prop_calc_2_dt = prop_calc(final_data, .2, 'data_type')
prop_calc_5_dt = prop_calc(final_data, .5, 'data_type')

In [35]:
prop_calc_2_dt.to_csv("topics/prop_calc_2_dt_final_model.csv")
prop_calc_5_dt.to_csv("topics/prop_calc_5_dt_final_model.csv")

In [116]:
prop_calc_2_dt

,data_type,total_score
0,Police Incident Report,7843.605297
1,Auto Collision Report,6220.267259
2,"Parcel Records, Permits, Plans",4488.424044
3,Crime Photo and Video,3767.362125
4,Criminal Record Check,3201.082004
5,"Environmental Assessment, Hazardous Materials",1882.571488
6,911/Law Enforcement Service Calls,1577.636194
7,"City Emails, Social Media",1525.486415
8,Property Liens,1424.262950
9,Building Code Violations,1371.284014


## Normalize Metrics within a City/County (Dampened Popularity):
* For each city/county, we add up total score fore each topic and then take the log of the total score. We then add up scores across each city/count.
* For winner-take-all, only score for top topic included (provided it is above threshold)
* For partial-credit, scores for all topics included (provided it is above threhsold
* This is an extra control for cities with a large number of PRRs from skewing our results


In [4]:
city_list = ['Arlington', 'Bainbridge', 'Boulder', 'CathedralCity' ,'Clearwater', 'Dayton', 
            'Denton', 'Everett', 'FortCollins', 'Greensboro', 'Hayward', 'Kirkland', 'LasCruces', 'Lynnwood',
            'Mercer', 'Miami', 'Middleborough', 'Nola', 'Oakland', 'OKC', 'Olympia', 'PaloAlto', 
            'Peoria', 'Pullman', 'RanchoCucamonga', 'Redmond', 'Renton', 'Sacramento', 'SanFrancisco', 
            'Tukwila', 'Vallejo', 'WestSacramento', 'Winchester']

In [11]:
def norm_pop(df, city_list, num_topics, thresh, winner_take_all, policy):
    
    if policy:
        list_dicts_1 = []
        list_dicts_0 = []
        
    else:
        list_of_domain_dicts = []
    
    for city in city_list:
        if policy:
            results_dict_1 = {}
            results_dict_0 = {}
            for num in range(0, num_topics):
                results_dict_1[num] = 0
                results_dict_0[num] = 0
        else:
            results_dict = {}
            for num in range(0, num_topics):
                results_dict[num] = 0
        
        #get our df only of rows from a given city/state domain
        city_df = df[df.city == city]
        
        for row_num in city_df.index:
            tup_list = city_df.topic_comp[row_num] #list of (topic, doc composition) tuples
            
            if winner_take_all:
        
                #return only the tuple w/highest topic composition value
                winner_tuple = max(tup_list, key=lambda item:item[1]) 
                if winner_tuple[1] > thresh:
                    if policy:
                        if city_df.policy[row_num] == 1:
                            results_dict_1[winner_tuple[0]] += winner_tuple[1]
                                
                        else:
                            results_dict_0[winner_tuple[0]] += winner_tuple[1]
                        
                        
                    else:
                        results_dict[winner_tuple[0]] += winner_tuple[1] 
                            
            else:
                if policy:
                    if city_df.policy[row_num] == 1:
                        for tup in tup_list: 
                            if tup[1] >= thresh:
                                results_dict_1[tup[0]] += tup[1] 
                    else:
                         for tup in tup_list: 
                            if tup[1] >= thresh:
                                results_dict_0[tup[0]] += tup[1]
                else:
                    
                    for tup in tup_list: 
                        if tup[1] >= thresh:
                            results_dict[tup[0]] += tup[1] 
            
        #when loop of domain_df is finished, take log of all keys in dict
        if policy:
            log_dict_1 = {}
            for k,v in results_dict_1.items():
                if v != 0:
                    log_dict_1[k] = np.log(v) 
                else:
                    log_dict_1[k] = 0
                
            log_dict_0 = {}
            for k,v in results_dict_0.items():
                if v != 0:
                    log_dict_0[k] = np.log(v) 
                else:
                    log_dict_0[k] = 0
        else:
            log_dict = {}
            for k,v in results_dict.items():
                if v != 0:
                    log_dict[k] = np.log(v) 
                else:
                    log_dict[k] = 0

        if policy:
            list_dicts_1.append(log_dict_1)
            list_dicts_0.append(log_dict_0)
        else:
            #now we have a polished dict of topic numbers as keys and log of all views/DLs as values; append it to list
            list_of_domain_dicts.append(log_dict)
    
    #use Counter() object to sync our dictionaries
    if policy:
        c1 = Counter()
        for d1 in list_dicts_1:
            c1.update(d1)
        c0 = Counter()
        for d0 in list_dicts_0:
            c0.update(d0)
    else:  
        c = Counter()
        for d in list_of_domain_dicts:
            c.update(d)

    if policy:
        pd1 = dict(c1)
        pd0 = dict(c0)
    else:
        popularity_dict = dict(c)
    
    if policy:
        pd_df1 = pd.DataFrame.from_dict(pd1, orient = 'index')
        pd_df1.reset_index(inplace = True)
        pd_df1['policy'] = 1
        pd_df1.rename(index = str, columns = {'index': 'topic', 0: 'total_score'}, inplace = True)
        
        pd_df1.topic = pd.to_numeric(pd_df1.topic)
        pd_df1 = pd_df1.sort_values(by=['topic'], ascending = True)

        # add in topic words
        words_in_topics = [tup[1] for tup in final_model.show_topics(num_topics=60, formatted=False)] # update w/ winning model
        pd_df1['topic_words'] = words_in_topics

        pd_df1[["topic1", "topic2", "topic3", "topic4", 
               "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = pd_df1.topic_words.apply(pd.Series)
        
        pd_df0 = pd.DataFrame.from_dict(pd0, orient = 'index')
        pd_df0.reset_index(inplace = True)
        pd_df0['policy'] = 0
        pd_df0.rename(index = str, columns = {'index': 'topic', 0: 'total_score'}, inplace = True)
        pd_df0.topic = pd.to_numeric(pd_df0.topic)
        pd_df0 = pd_df0.sort_values(by=['topic'], ascending = True)

        # add in topic words
        words_in_topics = [tup[1] for tup in final_model.show_topics(num_topics=60, formatted=False)] # update w/ winning model
        pd_df0['topic_words'] = words_in_topics

        pd_df0[["topic1", "topic2", "topic3", "topic4", 
               "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = pd_df0.topic_words.apply(pd.Series)
    
        
        pd_df = pd.concat([pd_df0, pd_df1])
    else:
        pd_df = pd.DataFrame.from_dict(popularity_dict, orient = 'index')
        pd_df.reset_index(inplace = True)
    
        pd_df.rename(index = str, columns = {'index': 'topic', 0: 'total_score'}, inplace = True)
        pd_df.topic = pd.to_numeric(pd_df.topic)
        pd_df = pd_df.sort_values(by=['topic'], ascending = True)

        # add in topic words
        words_in_topics = [tup[1] for tup in final_model.show_topics(num_topics=60, formatted=False)] # update w/ winning model
        pd_df['topic_words'] = words_in_topics

        pd_df[["topic1", "topic2", "topic3", "topic4", 
               "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = pd_df.topic_words.apply(pd.Series)



    pd_df = pd_df.sort_values(by='total_score', ascending=False)

    return pd_df

We remove observations from June 2018 since we obtained data from the cities in our sample at different points throughout the month of June. Because we are looking at differences between the treatment and control groups, we do not want differences in date different cities' data were accessed to influence our results.

In [14]:
pol_df = final_data[final_data['month_year'] != '2018-06-01']

In [ ]:
# calculates dampened popularity of data types from data frame with topic dampened popularity

def np_topic_to_data(np_df, policy, data_type_dict = data_type_dict):
    np_df['data_type'] = np_df['topic'].apply(lambda x: data_type_dict[x])
    if policy:
        np_gp = np_df.groupby(['data_type', 'policy']).sum()
    else:
        np_gp = np_df.groupby('data_type').sum()
    np_gp.reset_index(inplace= True)
    np_gp.sort_values(by = 'total_score', inplace = True, ascending = False)
    np_gp.reset_index(drop = True, inplace = True)
    np_gp.drop(columns=['topic'], inplace = True)
    return np_gp

In [18]:
# winner take all with different thresholds for topic and data types
np_df2 = norm_pop(pol_df, city_list, 60, .2, True, False)
np_df5 = norm_pop(pol_df, city_list, 60, .5, True, False)
np_df2.to_csv('topics/norm_pop_2_final.csv')
np_df5.to_csv('topics/norm_pop_5_final.csv')
np_data_type2 = np_topic_to_data(np_df2, False)
np_data_type5 = np_topic_to_data(np_df5, False)
np_data_type2.to_csv('topics/norm_pop_2_dt_final.csv')
np_data_type5.to_csv('topics/norm_pop_5_dt_final.csv')

In [16]:
# proportional calculation with different thresholds for topic and data types
np_df2_prop = norm_pop(pol_df, city_list, 60, .2, False, False)
np_df5_prop = norm_pop(pol_df, city_list, 60, .5, False, False)
np_df2_prop.to_csv('topics/norm_pop_2_prop_final.csv')
np_df5_prop.to_csv('topics/norm_pop_5_prop_final.csv')
np_data_type2_prop = np_topic_to_data(np_df2_prop, False)
np_data_type5_prop = np_topic_to_data(np_df5_prop, False)
np_data_type2_prop.to_csv('topics/norm_pop_2_dt_prop_final.csv')
np_data_type5_prop.to_csv('topics/norm_pop_5_dt_prop_final.csv')

In [18]:
# difference in topics w/ policy and proportional count
np_df2_prop = norm_pop(pol_df, city_list, 60, .2, False, True)
np_data_type2_prop = np_topic_to_data(np_df2_prop, True)
np_data_type2_prop.to_csv('topics/norm_pop_2_dt_prop_pol_final.csv')

In [71]:
dt_policy = norm_pop(pol_df, city_list, 60, .2, True, True)

In [ ]:
dt_policy = np_topic_to_data_type(dt_policy, True)
dt_policy.to_csv('topics/dt_policy.csv')

In [20]:
city_list_sub = ['Greensboro', 'NewOrleans', 'FortCollins']

In [22]:
pol_df_sub = pol_df[pol_df['city'].isin(city_list_sub)]

In [23]:
np_df2_sub = norm_pop(pol_df_sub, city_list, 60, .2, True, True)
np_data_type2_sub = np_topic_to_data(np_df2_sub, True)
np_data_type2_sub.to_csv('topics/norm_pop_2_dt_sub_pol_final.csv')

In [24]:
for city in city_list_sub:
    pol_df_city = pol_df[pol_df['city'] == city]
    np_df2_city = norm_pop(pol_df_city, [city], 60, .2, True, True)
    np_data_type2_city = np_topic_to_data(np_df2_city, True)
    fp = 'topics/norm_pop_2_dt_sub_pol_{}.csv'.format(city)
    np_data_type2_city.to_csv(fp)

## Topic Popularity by City

In [10]:
def pop_by_city(df, city_list, num_topics, thresh, winner_take_all):
    
    
    cols = ['city', 'topic', 'total_pop', 'pct'] 
    
    topic_pop_city = pd.DataFrame(columns = cols)
    
    for city in city_list:
        
        results_dict = {}
        for i in range(0, num_topics):
            results_dict[i] = 0
        
        #get our df only of rows from a given city/state domain
        city_df = df[df.city == city]
        
        for row_num in city_df.index:
            tup_list = city_df.topic_comp[row_num] #list of (topic, doc composition) tuples
            
            if winner_take_all:
        
                #return only the tuple w/highest topic composition value
                winner_tuple = max(tup_list, key=lambda item:item[1])  

                if not winner_tuple[0] in results_dict: #if not in dict, add it with its TOTAL VIEWS score
                    if winner_tuple[1] > thresh:
                        results_dict[winner_tuple[0]] = winner_tuple[1] 
                else:
                    pass

                if winner_tuple[0] in results_dict: #if in dict, increment that key's value with score
                    if winner_tuple[1] > thresh:
                        results_dict[winner_tuple[0]] += winner_tuple[1]
                    pass
            else:
                for tup in tup_list: 
                    if not tup[0] in results_dict:
                        if tup[1] >= thresh: 
                            results_dict[tup[0]] = tup[1] 
                    else:
                        pass
                    if tup[0] in results_dict:
                        if tup[1] >= thresh:
                            results_dict[tup[0]] += tup[1] 
        
        pd_df = pd.DataFrame.from_dict(results_dict, orient = 'index')
        pd_df.reset_index(inplace = True)
        pd_df.rename(index = str, columns = {'index': 'topic', 0: 'total_pop'}, inplace = True)
        pd_df['city'] = city
        pd_df['pct'] = (pd_df['total_pop']/sum(pd_df['total_pop']))*100
        topic_pop_city = pd.concat([topic_pop_city, pd_df])

    return topic_pop_city

In [11]:
pbc = pop_by_city(final_data, city_list, 60, .2, False)

In [12]:
pbc['data_type'] = pbc['topic'].apply(lambda x: data_type_dict[x])

In [12]:
pbc.to_csv('pbc.csv')

In [15]:
pbc_gb = pbc[pbc['city'] == "Greensboro"]
pbc_gb.sort_values(by = 'total_pop', ascending = False, inplace = True)
pbc_gb['topic'].iloc[0]

46

In [17]:
topic_set = set()
dt_set = set()

for city in city_list:
    pbc_city = pbc[pbc['city'] == city]
    pbc_city.sort_values(by = 'total_pop', ascending = False, inplace = True)
    topic_set.add(pbc_city['topic'].iloc[0])
    fp = 'topics/topic_pop_{}.csv'.format(city)
    pbc_city.to_csv(fp)
    
    pbc_city_gp = pbc_city.groupby('data_type').sum()
    pbc_city_gp.sort_values(by = 'total_pop', ascending = False, inplace = True)
    pbc_city_gp.reset_index(inplace=True)
    dt_set.add(pbc_city_gp['data_type'].iloc[0])
    fp2 = 'topics/data_type_pop_{}.csv'.format(city)
    pbc_city_gp.to_csv(fp2)

In [18]:
len(topic_set)

14

In [19]:
len(dt_set)

7

## City Deep-Dive

In [25]:
pol_df.to_csv('pol_df.csv')

In [17]:
city_dd = ['Clearwater','Greensboro', 'FortCollins', 'Nola']

for city in city_dd:
    city_df = final_df[final_df['city'] == city]
    month_count = city_df.groupby(['month_year', 'top_topic']).sum()
    month_count = month_count.reset_index()
    month_count = month_count[['month_year', 'top_topic', 'top_topic_comp']]
   
    fn = '{}_topic.csv'.format(city)
    month_count.to_csv(fn)
    
    df = month_count.groupby('top_topic').sum()
    df.sort_values(by = 'top_topic_comp', ascending = False, inplace = True)
    
    fn_tt = '{}_top_topic.csv'.format(city)
    df.to_csv(fn_tt)